# Import necessary Lib

In [ ]:
import sys, shutil
EssCod_PATH='/home/yuzhang/XPCS_Anlysis/notebook_XPCS/EssCode/'
EssCod_PATH in sys.path or sys.path.append(EssCod_PATH)
 

from Get_Data import readframe_series
from Process_Data import average_img, cpdump,cpopen 
from Plot_Data import plot,show_img,show_mask,show_img_with_ROI,pd_data_plot
from XPCS_Functions import make_qlist, calqlist,get_trace_and_maxc
from XPCS_Functions import get_pixellist_intensity,azimuthal_integration
from XPCS_Functions import calqlist_regions, get_waterfall
from XPCS import xpcs

from XSVS import xsvs
from Plot_XSVS import xsvs_plot_histogram


import numpy as np
import matplotlib.pyplot as plt


from Setup_file import *

In [ ]:
%matplotlib notebook
%matplotlib inline

# Sample description

sid = 'series4M_11_master.h5' 
FILENAME = 'coralPor_10Hz_'  
noframes = 1000
exposuretime= 100 ms
acquisition period =100 ms

# Load Data, Do average

In [ ]:
imgs = readframe_series(sid,inDir)

if T:
    ave = average_img(imgs)
    np.save( outDir + 'sid_%s-ave'%sid,ave)
if F:
    ave= np.load(outDir + 'sid_%s-ave.npy'%sid)

In [ ]:
imgs

In [ ]:
img=ave
img.shape


# Show images, data/mask/ROI

In [ ]:
show_img( ave, vmin=0.1,vmax=20,logs=True, set_cb=True,
         xlim=None,ylim=None,title='sid_%s-ave'%sid,
          save=True, outDir=outDir)

In [ ]:
show_mask( img,mask,vmin=0.1,vmax=20,
        logs=True, set_cb=True,
         xlim=None,ylim=None,title='sid_%save-mask'%sid,
          save=True, outDir=outDir)

In [ ]:
dimy,dimx = img.shape
qlist,qradi = make_qlist(qstart,qend,qwidth,noqs)
pixellist,qind,nopr,nopixels = calqlist(qlist,qradi, dimx,dimy,
                        cenx,ceny,qmask=mask); #calculate the pixellist, q_arrays

In [ ]:

title='sid_%s_ave-cuts_'%sid 
show_img_with_ROI(img, pixellist,qind+0, 0.01,1000,
                  save=True,outDir= outDir,title=title,set_cb=True)

# Plot one ring to check center

In [ ]:
cd XPCS_Anlysis/notebook_XPCS/EssCode/

In [ ]:
%run XPCS_Functions.py

In [ ]:

center =  [838.6 , 1634.66]
px, iy = get_pixellist_intensity( img, center, q=245, qwidth=2, qmask=mask)
plot(px,iy,logy=F,title='sid_%s-ave-Inten@q=245'%sid,xlabel='pixel',
         ylabel='intensity',xlim=None,outDir=outDir,save=True)

In [ ]:
center =  [838.6 , 1634.66]
px, iy = get_pixellist_intensity( img, center, q=120, qwidth=2, qmask=mask)
plot(px,iy,logy=F,title='sid_%s-ave-Inten@q=245'%sid,xlabel='pixel',
         ylabel='intensity',xlim=None,outDir=outDir,save=True)



# Plot q-Iq

In [ ]:
dimx,dimy

In [ ]:
%run XPCS_Functions.py
%run Process_Data.py

In [ ]:
    dimy,dimx = img.shape
 
    noqs = int( float(qend - qstart)/qwidth +1)
    qend = qstart + noqs*qwidth
    #print noqs,qend
    qlist,qradi = make_qlist(qstart,qend, qwidth, noqs)
    pixellist,qind,nopr,nopixels = calqlist(qlist,qradi, dimx,dimy, cenx,ceny,qmask)

In [ ]:
y, x = np.indices( [dimy,dimx] )
#y_= y- ceny +1;x_=x-cenx+1        

y_= y- ceny +1 ;x_=x-cenx +1 
r= np.int_( np.hypot(x_, y_)    + 0.5  )

#r= np.int_( np.hypot(x_, y_)    + 0.5  )
    
r= r.flatten() 
#print qlist
noqrs = len(qlist)    
qind = np.digitize(r, qlist)   

In [ ]:
y_[1]

In [ ]:
ceny

In [ ]:
ceny

In [ ]:
y_[0]

In [ ]:
qind[:10]

In [ ]:
r[:10]

In [ ]:
qlist[:10]

In [ ]:
qind.shape

In [ ]:
dimx

In [ ]:
qlist[100:120]

In [ ]:
qradi[100:120]

In [ ]:
if T:
    q,iq=azimuthal_integration( img, qstart=1, qend=1500, qwidth=1,
            cenx=cenx,ceny=ceny, qmask=mask,)
    cpdump([q,iq],'sid_%s-q-iq'%sid,outDir)
else:
    q,iq=cpopen('sid_%s-q-iq'%sid,outDir )

In [ ]:
 
plot(q*qperpixel,iq,logy=T,title='sid-%s_ave'%sid,xlabel='q',
         ylabel='Iq',xlim=[0.001,.12],outDir=outDir,save=True)